In [15]:
# import pandas as pd
# from jarvis.db.jsonutils import dumpjson
# df = pd.read_csv('https://raw.githubusercontent.com/vstanev1/Supercon/master/Supercon_data.csv')
# df=df.sample(frac=1)
# df['id']=df.index
# mem=[]
# for i,ii in df.iterrows():
#     info={}
#     info['id']=ii['id']
#     info['Tc']=ii['Tc']
#     info['formula']=ii['name']
#     mem.append(info)
# dumpjson(filename='supercon_chem.json',data=mem)

# test_size = 0.2
# n_train = int(len(df)*(1-(test_size)))
# n_test = len(df)-n_train

# #for leaderboard
# mem={}
# train={}

# test={}
# train_df = df[:n_train]

# test_df = df[-n_test:]
# for i,ii in train_df.iterrows():
#     train[str(ii['id'])]=ii['Tc']

# for i,ii in test_df.iterrows():
#     test[str(ii['id'])]=ii['Tc']
# mem['train']=train
# mem['test']=test
# fname='supercon_chem_Tc.json'
# dumpjson(data=mem,filename=fname)

In [16]:
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="supercon_chem.json",url="https://figshare.com/ndownloader/files/40719260")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229k/229k [00:00<00:00, 412kiB/s]


Loading the zipfile...
Loading completed.


In [17]:
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon
import pandas as pd
df = pd.DataFrame(dat)

In [18]:
df

,id,Tc,formula
0,1524,0.00,Tb1Th3.4Ru0.6B4
1,14309,0.00,B2Th1
2,6607,1.09,Ni2.5Cu1.5Zr8
3,4010,24.60,Pr0.91La1Ce0.09Cu1O4
4,14511,55.00,Pb0.5Ca1Sr2Y0.5Cu2O6.97
...,...,...,...
16409,442,0.00,As1Pd7
16410,14942,0.00,La1.5Gd1.5Ca1Ba2Sr1Cu6.3In0.7O16.62
16411,3978,0.00,N0.98Pr1
16412,1078,32.00,Sm0.1La1.7Sr0.2Cu1O4


In [19]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/supercon_chem_Tc.json.zip"
js_tag = "supercon_chem_Tc.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [20]:
id_data.keys()

dict_keys(['train', 'test'])

In [21]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [22]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [24]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['Tc'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['Tc'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
lgbm.fit(X_train,y_train)
pred=lgbm.predict(X_test)
print (mean_absolute_error(y_test,pred))

5.292053422775229
CPU times: user 1h 29min 58s, sys: 1.47 s, total: 1h 30min
Wall time: 5min 39s


In [25]:
from sklearn.metrics import r2_score
print(r2_score(y_test,pred))

0.8377923217446582


In [26]:
import os
f=open('AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv.zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv (deflated 50%)


0